# 1.Pré tratamento dos dados para análise.

In [151]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [152]:
df=pd.read_csv(r"Cuspidão\Completao.csv",delimiter=';',encoding="latin-1",decimal=',')

# Criar variáveis gerais

In [153]:
#Cria um variável com o nome de todos os setores do dataset e cria uma flag para se tem somente essa empresa no setor
ListofCompanies=df.drop_duplicates(subset=['SETOR_ATIV','Nome Empresa'])
ListofCompanies=ListofCompanies['SETOR_ATIV'].value_counts()
ListofCompanies=pd.DataFrame(ListofCompanies)
Listof1Companies=ListofCompanies[ListofCompanies['SETOR_ATIV']<4]
ListofCompanies=ListofCompanies[ListofCompanies['SETOR_ATIV']>=4]
ListofCompanies['Flag1Company']=0
Listof1Companies['Flag1Company']=1
ListofCompanies=pd.concat([ListofCompanies,Listof1Companies])
ListofCompanies['SETOR_ATIV']=ListofCompanies.index

#Cria Variáveis de Cuspir
DF_Polyline=pd.DataFrame()
DF_Cuspidao=pd.DataFrame()
DF_Cuspidao2=pd.DataFrame()
DF_Cuspidao_BAU=pd.DataFrame()
DF_Cuspidao_RESTO=pd.DataFrame()
df_produ_kmeans_REST_ALL=pd.DataFrame()

# Pré tratar dados para modelos


In [154]:
#Criar ano e mes
df['Ano']=df['DT_REFER'].apply(str).str[0:4]
df['Mes']=df['DT_REFER'].apply(str).str[5:7]
#Transformar colunas negativas em positivas
df['Passivo Circulante']=df['Passivo Circulante']*-1
df['Passivo Total']=df['Passivo Total']*-1
#Coloca a Flag de 1 company no dataset central
df1 = pd.merge( ListofCompanies,df, left_on='SETOR_ATIV', right_on='SETOR_ATIV')


In [155]:
#Transforma as relações de indicadores para clusterizar em numeros, para o for andar de lado.
df_produ[14]=df_produ['Ativo Circulante'].clip(lower=0)
df_produ[13]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[12]=df_produ['Passivo Circulante'].clip(upper=0)
df_produ[11]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[10]=df_produ['Passivo Total'].clip(upper=0)
df_produ[9]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[8]=df_produ['Ativo Total'].clip(lower=0)
df_produ[7]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[6]=df_produ['Reservas de Lucro'].clip(lower=0)
df_produ[5]=df_produ['Receita'].clip(lower=0)
df_produ[4]=df_produ['Receita'].clip(lower=0)
df_produ[3]=df_produ['Patrimônio Líquido'].clip(lower=0)
df_produ[2]=df_produ['Receita'].clip(lower=0)
df_produ[1]=df_produ['Lucro/Prejuízo do Período'].clip(lower=0)

# Clusterizar as empresas com dados realizados

In [156]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#BAU KMEANS
for y in range(1,13):
    for x in ListofCompanies.index.values:
        if y==1:
            y1=1
            y2=2
        else:
            y1=y+1
            y2=y+2
        #filtra o setor
        df_produ_kmeans=df_provisorio[(df_provisorio['SETOR_ATIV']==x)&(df_provisorio['Flag1Company']==0)].reset_index(drop=True)
        AnoMAX=df_produ_kmeans['DT_REFER'].apply(str).str[0:4].max()
        MesMAX=df_produ_kmeans['DT_REFER'].apply(str).str[5:7].max()
        #Filtrar ano e mês (ultimo tri de cada setor)
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)      
        #Clusteriza as empresas com valores negativos como 0 (ruim)
        df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
        df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
        #Separa as empresas que são elegíveis a escalabilidade e a clusterização
        df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
        df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
        
        #se tiver mais de 1 amostra (empresa)
        if len(df_produ_kmeans)>=2:
            #FAÇO A ESCALABILIDADE COM STANDARD
            scaler = StandardScaler() 
            pca = PCA(n_components = 2) 
            features= df_produ_kmeans[[y1,y2]]
            scaled_df = scaler.fit_transform(features) 
            normalized_df = normalize(scaled_df) 
            normalized_df = pd.DataFrame(normalized_df)
            #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
            X_principal = pca.fit_transform(normalized_df) 
            features = pd.DataFrame(X_principal) 
            features.columns = ['P1', 'P2']
            c=3
            if len(features)<3:
                c=len(features)
                
            #DOU UM FIT NO KMEANS
            kmeans = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0)
            kmeans.fit(features)
            #DPS EU FAÇO A CLUSTERIZAÇÃO
            d = KMeans(n_clusters=c, init='random',
                        n_init=10,
                        max_iter=300,random_state=0).fit_predict(features)
            df_produ_kmeans[f"{y1}{y2} Cluster"]=d
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
        else:
            DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
            DF_Cuspidao_BAU=pd.concat([DF_Cuspidao2,DF_Cuspidao_BAU]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_BAU
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_BAU=pd.DataFrame()
    
DF_Cuspidao_BAU=df_provisorio
DF_Cuspidao=pd.DataFrame()
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#REJEITADOS KMEANS
for y in range(1,13):
    if y==1:
        y1=1
        y2=2
    else:
        y1=y+1
        y2=y+2
    #filtra o setor
    df_produ_kmeans=df_provisorio[(df_provisorio['Flag1Company']==1)].reset_index(drop=True)
    AnoMAX=df_produ_kmeans['DT_REFER'].apply(str).str[0:4].max()
    MesMAX=df_produ_kmeans['DT_REFER'].apply(str).str[5:7].max()
    #Filtrar ano e mês (ultimo tri de cada setor)
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans["Ano"]==AnoMAX)&(df_produ_kmeans["Mes"]==MesMAX)].reset_index(drop=True)
    #Clusteriza as empresas com valores negativos como 0 (ruim)
    df_produ_kmeans_REST_NO=df_produ_kmeans[(df_produ_kmeans[y1]==0)|(df_produ_kmeans[y2]==0)].reset_index(drop=True)
    df_produ_kmeans_REST_NO[f"{y1}{y2} Cluster"]=0
    #Separa as empresas que são elegíveis a escalabilidade e a clusterização
    df_produ_kmeans=df_produ_kmeans[(df_produ_kmeans[y1]!=0)&(df_produ_kmeans[y2]!=0)].reset_index(drop=True)
    df_produ_kmeans=df_produ_kmeans.sort_values(by=[y1,y2]).reset_index(drop=True)
    #se tiver mais de 1 amostra (empresa)
    if len(df_produ_kmeans)>=2:
        #FAÇO A ESCALABILIDADE COM STANDARD
        scaler = StandardScaler() 
        pca = PCA(n_components = 2) 
        features= df_produ_kmeans[[y1,y2]]
        scaled_df = scaler.fit_transform(features) 
        normalized_df = normalize(scaled_df) 
        normalized_df = pd.DataFrame(normalized_df)
        #E DEPOIS REDUZO AS DIMENSÕES E CORRELAÇÕES COM O PCA
        X_principal = pca.fit_transform(normalized_df) 
        features = pd.DataFrame(X_principal) 
        features.columns = ['P1', 'P2']
        c=3
        if len(features)<3:
            c=len(features)
        #DOU UM FIT NO KMEANS
        kmeans = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0)
        kmeans.fit(features)
        #DPS EU FAÇO A CLUSTERIZAÇÃO
        d = KMeans(n_clusters=c, init='random',
                    n_init=10,
                    max_iter=300,random_state=0).fit_predict(features)
        df_produ_kmeans[f"{y1}{y2} Cluster"]=d
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    else:
        DF_Cuspidao2=pd.concat([df_produ_kmeans,df_produ_kmeans_REST_NO]).reset_index(drop=True)
        DF_Cuspidao_RESTO=pd.concat([DF_Cuspidao2,DF_Cuspidao_RESTO]).reset_index(drop=True)
    df_provisorio=DF_Cuspidao_RESTO
    DF_Cuspidao2=pd.DataFrame()
    DF_Cuspidao_RESTO=pd.DataFrame()
DF_kmeans=pd.concat([DF_Cuspidao_BAU,df_provisorio]).reset_index(drop=True)
DF_kmeans["Dados"]="Realizado"

In [157]:
DF_kmeans=DF_kmeans[['Nome Empresa','Ticker','SETOR_ATIV','DT_REFER',"Dados",
             1,2,3,4,5,6,7,8,9,10,11,12,13,14,
            '12 Cluster',
            '34 Cluster',
            '56 Cluster',
            '78 Cluster',
            '910 Cluster',
            '1112 Cluster',
            '1314 Cluster',
             ]]

# Prever os próximos 4 tri

In [158]:
#Separo os indicadores que eu quero prever
df_produ[11]=df_produ['Obrigações Fiscais']
df_produ[10]=df_produ['Obrigações Sociais e Trabalhistas']
df_produ[9]=df_produ['Contas a Receber']
df_produ[8]=df_produ['Ativo Circulante']
df_produ[7]=df_produ['Passivo Circulante']
df_produ[6]=df_produ['Passivo Total']
df_produ[5]=df_produ['Ativo Total']
df_produ[4]=df_produ['Reservas de Lucro']
df_produ[3]=df_produ['Patrimônio Líquido']
df_produ[2]=df_produ['Receita']
df_produ[1]=df_produ['Lucro/Prejuízo do Período']

In [159]:
df_provisorio=df_produ
df_provisorio=df_provisorio[['Nome Empresa',
                             'Ticker',
                             'SETOR_ATIV',
                             'Flag1Company',
                             'DT_REFER',
                             'Ano',
                             'Mes',
                             1,2,3,4,5,6,7,8,9,10,11]]

#pego a lista de empresas para analisar
LitsofTickers=df_provisorio.drop_duplicates(subset='Ticker')
LitsofTickers=LitsofTickers['Ticker'].values
for x in LitsofTickers:
    DF_Poly=df_provisorio[df_provisorio['Ticker']==x].reset_index(drop=True)
    DF_Poly.sort_values(by=['DT_REFER']).reset_index(drop=True)
    #transformo os indexes em variáveis e a data em datetime
    DF_Poly["indexes"]=DF_Poly.index
    DF_Poly['DT_REFER']=pd.to_datetime(DF_Poly['DT_REFER'])
    #pego a ultima linha dessa empresa para pegar as informações principais para criar um novo dataset
    Lastmonth=DF_Poly.iloc[-1]['DT_REFER']
    Lastname=DF_Poly.iloc[-1]['Nome Empresa']
    Lastsector=DF_Poly.iloc[-1]['SETOR_ATIV']
    Lastnamesub=DF_Poly.iloc[-1]['Ticker']
    Lastindex=DF_Poly.iloc[-1]['indexes']
    #Crio as datas que eu quero ver la na frente,baseado na data do ultimo balanço
    Lastmonth1=Lastmonth+ relativedelta(months=+3)
    Lastmonth2=Lastmonth+ relativedelta(months=+6)
    Lastmonth3=Lastmonth+ relativedelta(months=+9)
    Lastmonth4=Lastmonth+ relativedelta(months=+12)
    #Crio um dataframe novo com essas datas
    Dateto=pd.DataFrame({'Data_REF': [Lastmonth1,Lastmonth2,Lastmonth3,Lastmonth4]})
    X_seq=pd.DataFrame({'x': [Lastindex+1,Lastindex+2,Lastindex+3,Lastindex+4]})
    #Configuro o modelo polimonial
    polyreg=make_pipeline(PolynomialFeatures(3),LinearRegression())
    Pred=Dateto
    Pred["Nome Empresa"]=Lastname
    Pred['Ticker']=Lastnamesub
    Pred['SETOR_ATIV']=Lastsector
    #Leio o historico de todos os indicadores e faço a previsão
    for y in range(1,12):
        yy=DF_Poly[['indexes']]
        xx=DF_Poly[[y]]
        polyreg.fit(yy,xx)
        Pred2=polyreg.predict(X_seq)
        Pred2=pd.DataFrame(Pred2,columns=[y])
        Pred[y]=Pred2
    DF_Polyline=pd.concat([Pred,DF_Polyline]).reset_index(drop=True)

    
DF_Polyline['Ano']=DF_Polyline['Data_REF'].apply(str).str[0:4]
DF_Polyline['Mes']=DF_Polyline['Data_REF'].apply(str).str[5:7]
DF_Polyline['Dados']="Previsão"

# Organizar os dados para cuspir no Front

In [160]:
DF_kmeans=DF_kmeans.fillna(0)
#Separo as colunas principais do dataset Kmeans e dou um replace nas clusterizações para criar uma nota
DF_kmeans_2=DF_kmeans.loc[:,('Ticker','DT_REFER',"Dados",'12 Cluster',
            '34 Cluster',
            '56 Cluster',
            '78 Cluster',
            '910 Cluster',
            '1112 Cluster',
            '1314 Cluster')].replace([0,1,2],[0,10,5])
#Crio a nota da empresa baseada nos clusters (0 é ruim, 1 é bom e 2 é mediano)
DF_kmeans_2['Nota da Empresa']=DF_kmeans_2['12 Cluster']+DF_kmeans_2['34 Cluster']+DF_kmeans_2['56 Cluster']+DF_kmeans_2['78 Cluster']+DF_kmeans_2['910 Cluster']+DF_kmeans_2['1112 Cluster']+DF_kmeans_2['1314 Cluster']
DF_kmeans_2['Nota da Empresa']=DF_kmeans_2['Nota da Empresa']/7
DF_kmeans_2['Desempenho do Trimestre']=np.where(DF_kmeans_2['Nota da Empresa']<5,"Ruim",
                                        (np.where((DF_kmeans_2['Nota da Empresa']>=5) & (DF_kmeans_2['Nota da Empresa']<7) ,"Médio","Bom")))
#Pego o indicador com a melhor nota de clusterização (indicador denominador)
DF_kmeans_2['Melhor Indicador1']=(DF_kmeans_2['12 Cluster']+DF_kmeans_2['34 Cluster']+DF_kmeans_2['56 Cluster'])/3
DF_kmeans_2['Melhor Indicador2']=(DF_kmeans_2['34 Cluster']+DF_kmeans_2['78 Cluster']+DF_kmeans_2['910 Cluster']+DF_kmeans_2['1112 Cluster']+DF_kmeans_2['1314 Cluster'])/5
DF_kmeans_2['Melhor Indicador']=np.where(DF_kmeans_2['Melhor Indicador1']>DF_kmeans_2['Melhor Indicador2'],"Receita",
                                        (np.where(DF_kmeans_2['Melhor Indicador1']<DF_kmeans_2['Melhor Indicador2'],"Patrimônio Líquido","Nenhum")))
df_produ=df1
#Pego as colunas antigas sem tratamento de clips
DF_kmeans_2 = pd.merge(df_produ,DF_kmeans_2 ,left_on=['Ticker','DT_REFER'], right_on=['Ticker','DT_REFER'])
DF_kmeans_2['Analise']=DF_kmeans_2['Flag1Company'].replace([0,1],["Analise por Setor",'Analise com outras empresas'])
DF_kmeans_2['Tipo de Analise']='Kmeans'
#Deixo tudo limpinho e arrumado no dataset kmeans
DF_kmeans_2=DF_kmeans_2[['Nome Empresa','Ticker','SETOR_ATIV','Dados','Analise','Melhor Indicador','Desempenho do Trimestre','DT_REFER','Lucro/Prejuízo do Período','Ativo Total','Ativo Circulante','Contas a Receber','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas','Obrigações Fiscais','Reservas de Lucro','Patrimônio Líquido']]
#Dou nome aos bois no dataset do polimonial
DF_Polyline_2=DF_Polyline.rename({'Data_REF': 'DT_REFER', 
                                  1: 'Lucro/Prejuízo do Período',
                                  2: 'Receita', 
                                  3: 'Patrimônio Líquido', 
                                  4: 'Reservas de Lucro', 
                                  5: 'Ativo Total',
                                  6: 'Passivo Total', 
                                  7: 'Passivo Circulante', 
                                  8: 'Ativo Circulante',
                                  9: 'Contas a Receber',
                                  10: 'Obrigações Sociais e Trabalhistas',
                                  11: 'Obrigações Fiscais',
                                 }, axis=1)
#Seto as mesmas colunas do kmeans
DF_Polyline_2['Melhor Indicador']='Não Analisado'
DF_Polyline_2['Desempenho do Trimestre']='Não Analisado'
DF_Polyline_2['Nota da Empresa']='Não Analisado'
DF_Polyline_2['Analise']='Analise de Historico Financeiro'
DF_Polyline_2['Tipo de Analise']='Regressão Polimonial'
#Deixo tudo limpinho e arrumado no dataset polimonial
DF_Polyline_2=DF_Polyline_2[['Nome Empresa','Ticker','SETOR_ATIV','Dados','Analise','Melhor Indicador','Desempenho do Trimestre','DT_REFER','Lucro/Prejuízo do Período','Ativo Total','Ativo Circulante','Contas a Receber','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas','Obrigações Fiscais','Reservas de Lucro','Patrimônio Líquido']]
#Deipois junto tudo e cuspo
Base_Unica=pd.concat([DF_Polyline_2,DF_kmeans_2]).reset_index(drop=True)
Base_Unica.to_csv(rf"Cuspidão\Base_Unica.csv",decimal=',',sep=';', encoding='latin-1',index=False)